In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output

from array import array

from datetime import datetime, timedelta, date #for time duration calculations
from dateutil.parser import parse #for fuzzy finding year

import pickle #for saving output files, pickles
from sys import stdout
import time #for time.sleep function to delay calls
from tqdm import tqdm #for updating loop
#from os import listdir
#from os.path import isfile, join
import glob #pattern matching and expansion.

## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.sql import table, column, select, update, insert
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import pandas as pd

#In Python: Define your username and password used above. I've defined the database name (we're 
#using a dataset on births, so I call it birth_db). 
dbname = 'donors_db'
username = 'russell'
pswd = 'bradypodion'

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print('postgresql://%s:%s@localhost/%s'%(username,pswd,dbname))
print(engine.url)
# Replace localhost with IP address if accessing a remote server

In [ ]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))
print(engine.url)


In [5]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [20]:
### query: from historical data
merge_query = """
SELECT * FROM merge_projects;
"""

mergedframe = pd.read_sql_query(merge_query,con)

In [23]:
mergedframe = mergedframe[mergedframe.funding_status == 'completed']
mergedframe.head(2)

,index,index_x,id,proposalURL,fundURL,imageURL,retinaImageURL,thumbImageURL,fulfillmentTrailer,percentFunded,numDonors,costToComplete,studentLed,numStudents,professionalDevelopment,distanceLearningProject,totalPrice,freeShipping,teacherId,teacherName,schoolName,schoolUrl,city,zip,state,stateFullName,latitude,longitude,expirationDate,expirationTime,fundingStatus,fullyFundedDate,waitingForCheckPayment,modifiedDate,shortDescription,matchingfund,grade_level_x,poverty_level_x,r_path,ffyear,index_y,_projectid,_teacher_acctid,_schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_prefix,teacher_teach_for_america,primary_focus_subject,primary_focus_area,secondary_focus_subject,secondary_focus_area,resource_type,poverty_level_y,grade_level_y,vendor_shipping_charges,sales_tax,payment_processing_charges,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,total_donations,num_donors,eligible_double_your_impact_match,eligible_almost_home_match,funding_status,date_posted,date_completed,date_thank_you_packet_mailed,date_expiration,calendar_completed,year_completed,calendar_expired
0,0,0,846450,https://www.donorschoose.org/project/the-gathe...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u48...,https://www.donorschoose.org/teacher/photo/u48...,https://www.donorschoose.org/teacher/photo/u48...,My students need a rug.,100,4,0.00,False,350,False,False,506.53,false,481042,Mr. Gamble,Sweet Water High School,https://www.donorschoose.org/school/sweet-wate...,Sweet Water,36782,AL,Alabama,32.094549,-87.871459,2013-01-24,1359003600000,funded,"Sunday, September 23, 2012 11:42:51 PM EDT",false,"Wednesday, February 13, 2013 12:00:00 AM EST",Our students need a place to sit. In our new m...,no,Grades PreK-2,MODERATEHIGH,https://www.donorschoose.org/common/json_feed....,2012,437358,32a7d6bce2dda94bfc214313672de865,5a7bbfcdb6bb9975fc070a7f29b6787e,c041ebd884b52071de26f560ac4c2f8d,1.022500e+10,32.094549,-87.871459,Sweet Water,AL,36782.0,rural,Marengo Co School District,Marengo,f,f,f,f,f,f,Mr.,f,Music,Music & The Arts,Performing Arts,Music & The Arts,Other,high poverty,Grades PreK-2,36.52,27.06,4.98,30.0,430.55,506.53,350.0,503.00,4,t,f,completed,2012-08-28 00:00:00,2012-09-23 00:00:00,2013-01-29 00:00:00,2013-01-24 00:00:00,2012-09-23,2012,2013-01-24
1,7,0,2116678,https://www.donorschoose.org/project/extreme-m...,https://secure.donorschoose.org/donors/givingC...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,https://www.donorschoose.org/teacher/photo/u10...,My students need storage cabinets in our choir...,100,4,0.00,False,92,False,False,1345.66,true,1030090,Mrs. Kenney,Beauregard High School,https://www.donorschoose.org/school/beauregard...,Opelika,36804-2064,AL,Alabama,32.551149,-85.375064,2016-12-08,1481173200000,funded,"Thursday, September 29, 2016 9:07:42 PM EDT",false,"Wednesday, November 30, 2016 12:00:00 AM EST",My choral classroom is a place where my studen...,no,Grades 9-12,MODERATEHIGH,https://www.donorschoose.org/common/json_feed....,2016,1118806,078269f7477e38767b3b7b63f85feb39,86e3438cb9089a69ca4b728e5e862422,aa186c0ded5256b3878be19ad9ad49a1,1.020700e+10,32.551149,-85.375064,Opelika,AL,36804.0,rural,Lee Co School District,Lee,f,f,f,f,f,f,Mrs.,f,Performing Arts,Music & The Arts,Music,Music & The Arts,Supplies,high poverty,Grades 9-12,0.00,82.79,15.24,30.0,1143.81,1345.66,92.0,1343.89,4,f,f,completed,2016-08-09 00:00:00,2016-09-29 00:00:00,2016-09-29 00:00:00,2016-12-08 00:00:00,2016-09-29,2016,2016-12-08


In [22]:
mergedframe['funding_status'].value_counts()

completed    82091
Name: funding_status, dtype: int64

In [15]:
mergedframe['poverty_level_y'].value_counts()

highest poverty     42068
high poverty        17083
high                13470
moderate poverty     6738
low                  1521
low poverty           931
minimal               179
unknown               126
Name: poverty_level_y, dtype: int64

In [12]:
#pd.mergedframe('display.max_columns', None)
list(mergedframe['fulfillmentTrailer'])
#mergedframe.head(2)

['My students need a rug.',
 'My students need storage cabinets in our choir room to house our music library.',
 'My students need 4 clarinets and 1 trumpet to play in the band.',
 'My students need 6 new stands at the high school in order to accommodate our rapidly growing program.',
 'My students need a projector to be able to view live performances and other teaching resources from the Internet.',
 'My students need black binders to hold our music for daily and concert use.',
 'My students need a wireless laptop   My students need a clean classroom environment   My students need 2 bar code scanners for fast check out in library',
 'My students need a set of 18 Bucket Drums and Stability Balls to become rhythmic wonders in the general music classroom!',
 'My students need updated rhythm instruments to replace the current supply, some of which are broken, and all of which are over 30 years old. They need bells, egg shakers, and rhythm sets.',
 'My students need a bell kit to perform t

In [ ]:
### Close communication with the database
con.close()